In [ ]:
#@title ##### License { display-mode: "form" }
# Copyright 2019 DeepMind Technologies Ltd. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# OpenSpiel: RRPS Example

* This Colab gets you started with installing OpenSpiel and its dependencies.
* OpenSpiel is a framework for reinforcement learning in games.
* For a longer intro to OpenSpiel, see [the tutorial video](https://www.youtube.com/watch?v=8NCPqtPwlFQ), [documentation](https://openspiel.readthedocs.io/en/latest/), or [API reference](https://openspiel.readthedocs.io/en/latest/api_reference.html).
* This colab also includes examples of how to get started with the Roshambo/RRPS environment and bots. It is based on [roshambo_population_example.py](https://github.com/google-deepmind/open_spiel/blob/master/open_spiel/python/examples/roshambo_population_example.py)
* For more info on Roshambo, see the [RRPS benchmark paper](https://openreview.net/pdf?id=gQnJ7ODIAx).

## Install

Install OpenSpiel via pip:

Note that if you are not using a colab, then you would use
python3 -m pip install open_spiel
Additional information about installing OpenSpiel can be found at the links above.


In [ ]:
!pip install --upgrade open_spiel

# Simple example of OpenSpiel API: uniform random trajectory on Tic-Tac-Toe

This example is not used for RRPS, but it shows how to load a game in OpenSpiel, how to access a game state, and implements a random strategy.

In [ ]:
import numpy as np
import pyspiel

game = pyspiel.load_game("tic_tac_toe")
state = game.new_initial_state()

while not state.is_terminal():
  state.apply_action(np.random.choice(state.legal_actions()))
  print(str(state) + '\n')

# Getting started with RRPS


In [ ]:
# Imports
import numpy as np

from open_spiel.python import rl_agent
from open_spiel.python import rl_environment
import pyspiel


In [ ]:
# Some helper classes and functions.
# DO NOT CHANGE.

class BotAgent(rl_agent.AbstractAgent):
  """Agent class that wraps a bot.

  Note, the environment must include the OpenSpiel state in its observations,
  which means it must have been created with use_full_state=True.

  This is a simple wrapper that lets the RPS bots be interpreted as agents under
  the RL API.
  """

  def __init__(self, num_actions, bot, name="bot_agent"):
    assert num_actions > 0
    self._bot = bot
    self._num_actions = num_actions

  def restart(self):
    self._bot.restart()

  def step(self, time_step, is_evaluation=False):
    # If it is the end of the episode, don't select an action.
    if time_step.last():
      return
    _, state = pyspiel.deserialize_game_and_state(
        time_step.observations["serialized_state"])
    action = self._bot.step(state)
    probs = np.zeros(self._num_actions)
    probs[action] = 1.0
    return rl_agent.StepOutput(action=action, probs=probs)


#  We will use this function to evaluate the agents. Do not change.

def eval_agents(env, agents, num_players, num_episodes, verbose=False):
  """Evaluate the agent.

  Runs a number of episodes and returns the average returns for each agent as
  a numpy array.

  Arguments:
    env: the RL environment,
    agents: a list of agents (size 2),
    num_players: number of players in the game (for RRPS, this is 2),
    num_episodes: number of evaluation episodes to run.
    verbose: whether to print updates after each episode.
  """
  sum_episode_rewards = np.zeros(num_players)
  for ep in range(num_episodes):
    for agent in agents:
      # Bots need to be restarted at the start of the episode.
      if hasattr(agent, "restart"):
        agent.restart()
    time_step = env.reset()
    episode_rewards = np.zeros(num_players)
    while not time_step.last():
      agents_output = [
          agent.step(time_step, is_evaluation=True) for agent in agents
      ]
      action_list = [agent_output.action for agent_output in agents_output]
      time_step = env.step(action_list)
      episode_rewards += time_step.rewards
    sum_episode_rewards += episode_rewards
    if verbose:
      print(f"Finished episode {ep}, "
            + f"avg returns: {sum_episode_rewards / (ep+1)}")

  return sum_episode_rewards / num_episodes


def print_roshambo_bot_names_and_ids(roshambo_bot_names):
  print("Roshambo bot population:")
  for i in range(len(roshambo_bot_names)):
    print(f"{i}: {roshambo_bot_names[i]}")

def create_roshambo_bot_agent(player_id, num_actions, bot_names, pop_id):
  name = bot_names[pop_id]
  # Creates an OpenSpiel bot with the default number of throws
  # (pyspiel.ROSHAMBO_NUM_THROWS). To create one for a different number of
  # throws per episode, add the number as the third argument here.
  bot = pyspiel.make_roshambo_bot(player_id, name)
  return BotAgent(num_actions, bot, name=name)


#The following functions are used to load the bots from the original RRPS competition.

In [ ]:
# Some basic info and initialize the population

# print(pyspiel.ROSHAMBO_NUM_BOTS)    # 43 bots
# print(pyspiel.ROSHAMBO_NUM_THROWS)  # 1000 steps per episode

# The recall is how many of the most recent actions are presented to the RL
# agents as part of their observations. Note: this is just for the RL agents
# like DQN etc... every bot has access to the full history.
RECALL = 20

# The population of 43 bots. See the RRPS paper for high-level descriptions of
# what each bot does.

print("Loading bot population...")
pop_size = pyspiel.ROSHAMBO_NUM_BOTS
print(f"Population size: {pop_size}")
roshambo_bot_names = pyspiel.roshambo_bot_names()
roshambo_bot_names.sort()
print_roshambo_bot_names_and_ids(roshambo_bot_names)

bot_id = 0
roshambo_bot_ids = {}
for name in roshambo_bot_names:
  roshambo_bot_ids[name] = bot_id
  bot_id += 1


#Example showing how to load to agents from the RRPS bot population and evalute them against each other.

In [ ]:
# Example: create an RL environment, and two agents from the bot population and
# evaluate these two agents head-to-head.

# Note that the include_full_state variable has to be enabled because the
# BotAgent needs access to the full state.
env = rl_environment.Environment(
    "repeated_game(stage_game=matrix_rps(),num_repetitions=" +
    f"{pyspiel.ROSHAMBO_NUM_THROWS}," +
    f"recall={RECALL})",
    include_full_state=True)
num_players = 2
num_actions = env.action_spec()["num_actions"]
# Learning agents might need this:
# info_state_size = env.observation_spec()["info_state"][0]

# Create two bot agents
p0_pop_id = 0   # actr_lag2_decay
p1_pop_id = 1   # adddriftbot2
agents = [
    create_roshambo_bot_agent(0, num_actions, roshambo_bot_names, p0_pop_id),
    create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, p1_pop_id)
]

print("Starting eval run.")
avg_eval_returns = eval_agents(env, agents, num_players, 10, verbose=True)

print("Avg return ", avg_eval_returns)

#Basic Template for an RL agent

Below is a template of a basic RL agent to get you started. You will be implementing the step() function.

In [ ]:
# Template to get you started. Pick one of these to implement as your starting
# point.

# Template : Basic RL agent.
#
#
class MyAgent(rl_agent.AbstractAgent):
  """Agent class that learns to play RRPS.

  You fill this in to create your RRPS agent.

  See the superclass for more info: https://github.com/google-deepmind/open_spiel/blob/master/open_spiel/python/rl_agent.py
  """

  def __init__(self, num_actions, name="bot_agent"):
    assert num_actions > 0
    self._num_actions = num_actions  # 3

  def step(self, time_step, is_evaluation=False):
    # If it is the end of the episode, don't select an action.
    if time_step.last():
      return
    # Note: If the environment was created with include_full_state=True, then
    # game and state can be obtained as follows:

    game, state = pyspiel.deserialize_game_and_state(time_step.observations["serialized_state"])

    # A useful piece of information is the history (previous actions taken by agents).
    # You can access this by state.history()

    # Do something here that selects an action and computes a policy
    # distribution in probs.
    if len(state.history())==0:
      action = 0
    else:
      action = state.history()[-1]
    probs = np.ones(self._num_actions) / self._num_actions
    return rl_agent.StepOutput(action=action, probs=probs)



#A short example to show how to evaluate your agent againt an RRPS bot.

In [13]:
# Just trying an example out.

my_agent = MyAgent(3, name="kate_agent")
print(my_agent._num_actions)


p1_pop_id = 1   # adddriftbot2
agents = [
    my_agent,
    create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, p1_pop_id)
]


print("Starting eval run.")
avg_eval_returns = eval_agents(env, agents, num_players, 10, verbose=True)

print("Avg return ", avg_eval_returns)

probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
Finished episode 3, avg returns: [ 11.5 -11.5]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.33333333 0.33333333 0.33333333]
probs [0.3

KeyboardInterrupt: 